# FOM - Anwendungsfelder der Business Analytics am Beispiel von Fraud Detection im von Kreditkartentransaktionen

In diesem Jupyter Notebook soll der nicht-überwachte K-Means Algorithmus für das vollautomatische Clustering von Kreditkartentransaktionen verwendet werden. Dieser K-Means Algortihmus soll auf den in einer Kreditkartentransaktion bezahlten Geldbetrag angewandt werden, sodass im Anschluss geprüft werden kann, zu welchem Cluter eine Kreditkarte gehört. Mit diesem Vorgehen kann das Verwendungsprofil einer Kreditkarte identifiziert werden.

<b>Für dieses Vorhaben werden folgende Schritte unternommen:</b>
1. Import aller benötigten Python Packages
2. Laden der Daten aus dem AWS S3 Bucket
3. 

Im folgenden wird mit dem Import alles benötigten Python Packages begonnen.

In [1]:
# import python packages required for k-means-clustering
import os
import io
import boto3
import pandas as pd
from sagemaker import KMeans, get_execution_role